In [ ]:
#@title Initial Package Installation
!apt-get install -y xvfb x11-utils
!pip install pyvirtualdisplay==0.2.*

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
x11-utils is already the newest version (7.7+5build2).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [ ]:
#@title Initial Libraries to Import
import gym
from pyvirtualdisplay import Display
from gym.wrappers.monitoring.video_recorder import VideoRecorder
from base64 import b64encode
from IPython.display import HTML

In [ ]:
#@title Making an original gym environment and resetting it appropriately
env = gym.make("CartPole-v1")
env.action_space
env.reset()

array([-0.02263485, -0.0462552 , -0.00545204,  0.00530956], dtype=float32)

In [ ]:
#@title Training an Agent By Steps
for i in range(15):

  observation, reward, done, info = env.step(env.action_space.sample())

  if done:
    env.reset()
    continue

  print("step", i, observation, reward, done, info)

step 0 [-0.02355995 -0.24129854 -0.00534585  0.29626733] 1.0 False {}
step 1 [-0.02838592 -0.04610079  0.00057949  0.00190321] 1.0 False {}
step 2 [-0.02930794  0.14901285  0.00061756 -0.2905968 ] 1.0 False {}
step 3 [-0.02632768  0.344126   -0.00519438 -0.58308494] 1.0 False {}
step 4 [-0.01944516  0.14907719 -0.01685608 -0.29204282] 1.0 False {}
step 5 [-0.01646362 -0.04580042 -0.02269693 -0.00472338] 1.0 False {}
step 6 [-0.01737962 -0.24058963 -0.0227914   0.28071287] 1.0 False {}
step 7 [-0.02219142 -0.43537918 -0.01717714  0.5661212 ] 1.0 False {}
step 8 [-0.030899   -0.630256   -0.00585472  0.8533435 ] 1.0 False {}
step 9 [-0.04350412 -0.43505475  0.01121215  0.5588253 ] 1.0 False {}
step 10 [-0.05220522 -0.63033223  0.02238866  0.8550195 ] 1.0 False {}
step 11 [-0.06481186 -0.43552247  0.03948905  0.56945974] 1.0 False {}
step 12 [-0.07352231 -0.63117534  0.05087824  0.87431705] 1.0 False {}
step 13 [-0.08614582 -0.8269507   0.06836458  1.1825519 ] 1.0 False {}
step 14 [-0.1026

In [ ]:
#@title Rendering a Visualization
display = Display(visible=False, size=(1400, 900))
_ = display.start()
before_training = "before_training.mp4"

#Recording Actions
video = VideoRecorder(env, before_training)
env.reset()
for i in range(200):
  env.render()
  video.capture_frame()
  observation, reward, done, info = env.step(env.action_space.sample())

#Creating HTML Video object
video.close()
env.close()
def render_mp4(videopath: str) -> str:
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

/usr/local/lib/python3.10/dist-packages/gym/envs/classic_control/cartpole.py:163: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  logger.warn(


In [ ]:
#@title Video Display
html = render_mp4(before_training)
HTML(html)

In [ ]:
#@title Second Package Installation
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install 'imageio==2.4.0'
!pip install tf-agents
!pip install pyglet

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,304 kB]
Fetched 1,533 kB in 2s (832 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Do

In [ ]:
#@title Final Libraries to Import
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import categorical_q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common


In [ ]:
#@title Hyperparamters to Tune
env_name = "CartPole-v1"
num_iterations = 2000

initial_collect_steps = 1000
collect_steps_per_iteration = 1
replay_buffer_capacity = 100000

fc_layer_params = (100,)

batch_size = 64
learning_rate = 1e-3
gamma = 0.99
log_interval = 200

num_atoms = 51
min_q_value = -20
max_q_value = 20
n_step_update = 2

num_eval_episodes = 10
eval_interval = 1000

In [ ]:
#@title New Gym Environment, New Categorical DQN (C51) Agent
train_py_env = suite_gym.load(env_name)
eval_py_env = suite_gym.load(env_name)

train_env = tf_py_environment.TFPyEnvironment(train_py_env)
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

#Network for for the agent (with optimizer)
categorical_q_net = categorical_q_network.CategoricalQNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    num_atoms=num_atoms,
    fc_layer_params=fc_layer_params)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

#Creating new agent
agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=categorical_q_net,
    optimizer=optimizer,
    min_q_value=min_q_value,
    max_q_value=max_q_value,
    n_step_update=n_step_update,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=gamma,
    train_step_counter=train_step_counter)
agent.initialize()

In [ ]:
#@title Calculating Average Return as a Relevant Metric and Choosing Random Policy
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):

    time_step = environment.reset()
    episode_return = 0.0

    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]


#Random Policy Choosing
random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

compute_avg_return(eval_env, random_policy, num_eval_episodes)

22.6

In [ ]:
#@title Creating a buffer to store data from previous experiences, collecting steps function and loading initial interaction
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=train_env.batch_size,
    max_length=replay_buffer_capacity)

#Collecting Step Function
def collect_step(environment, policy):
  time_step = environment.current_time_step()
  action_step = policy.action(time_step)
  next_time_step = environment.step(action_step.action)
  traj = trajectory.from_transition(time_step, action_step, next_time_step)
  replay_buffer.add_batch(traj)

#Loop to collect the initial interaction of agent
for _ in range(initial_collect_steps):
  collect_step(train_env, random_policy)
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, sample_batch_size=batch_size,
    num_steps=n_step_update + 1).prefetch(3)
iterator = iter(dataset)

In [ ]:
#@title Training the Agent
try:
  %%time
except:
  pass

agent.train = common.function(agent.train)
agent.train_step_counter.assign(0)
avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
returns = [avg_return]

for _ in range(num_iterations):

  # Collecting some few steps and saving them to the replay buffer
  for _ in range(collect_steps_per_iteration):
    collect_step(train_env, agent.collect_policy)

  # Using a sample batch of data from the buffer and updating the agent's network
  experience, unused_info = next(iterator)
  train_loss = agent.train(experience)

  step = agent.train_step_counter.numpy()

  if step % log_interval == 0:
    print('step = {0}: loss = {1}'.format(step, train_loss.loss))

  if step % eval_interval == 0:
    avg_return = compute_avg_return(eval_env, agent.policy, num_eval_episodes)
    print('step = {0}: Average Return = {1:.2f}'.format(step, avg_return))
    returns.append(avg_return)

step = 200: loss = 3.1252453327178955
step = 400: loss = 1.986607551574707
step = 600: loss = 1.8434069156646729
step = 800: loss = 1.6095060110092163
step = 1000: loss = 1.4459545612335205
step = 1000: Average Return = 180.90
step = 1200: loss = 1.3159339427947998
step = 1400: loss = 1.1550500392913818
step = 1600: loss = 1.0602624416351318
step = 1800: loss = 1.1311241388320923
step = 2000: loss = 0.9662829041481018
step = 2000: Average Return = 214.20


In [ ]:
#@title Rendering the Video
def embed_mp4(filename):
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)

#Using the steps and image.io to create a local file of the video
num_episodes = 3
video_filename = 'imageio.mp4'
with imageio.get_writer(video_filename, fps=60) as video:
  for _ in range(num_episodes):
    time_step = eval_env.reset()
    video.append_data(eval_py_env.render())
    while not time_step.is_last():
      action_step = agent.policy.action(time_step)
      time_step = eval_env.step(action_step.action)
      video.append_data(eval_py_env.render())

embed_mp4(video_filename)